# DeepFM

## Constantes

SEED = 42

## Importaciones

In [ ]:
!python -m pip install deepctr-torch torch pandas numpy scikit-learn recommenders

  Using cached tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl.metadata (1.1 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━ 484.4/899.8 MB 4.2 MB/s eta 0:01:40

In [ ]:
from recommenders.datasets import movielens
from deepctr_torch.inputs import SparseFeat, VarLenSparseFeat, get_feature_names
from deepctr_torch.models import DeepFM
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd